# Aligning non-manuscript source to target using Machine/SIL versification



In [1]:
#r "nuget:SIL.Scripture,7.0.0"
#r "nuget:Thot,3.3.5"
#r "../src/ClearBible.Engine/bin/Debug/net6.0/ClearBible.Engine.dll"
#r "../../machine/src/SIL.Machine/bin/Debug/netstandard2.0/SIL.Machine.dll"
#r "../../machine/src/SIL.Machine.Translation.Thot/bin/Debug/netstandard2.0/SIL.Machine.Translation.Thot.dll"

using SIL.Machine.Corpora;
using SIL.Machine.Tokenization;
using SIL.Machine.Translation;
using SIL.Machine.Translation.Thot;
using static SIL.Machine.Corpora.TokenProcessors;
using SIL.Machine.Utils;

using ClearBible.Engine.Translation;
using ClearBible.Engine.Corpora;
using ClearBible.Engine.Persistence;
using ClearBible.Engine.Tokenization;

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages SIL.Scripture, 7.0.0 Thot, 3.3.5

## Prepare corpora

In [2]:
var sourceCorpus = new ParatextTextCorpus("data/VBL-PT")
    .Tokenize<LatinWordTokenizer>();

var targetCorpus = new ParatextTextCorpus("data/WEB-PT")
    .Tokenize<LatinWordTokenizer>();

var parallelTextCorpus = sourceCorpus.EngineAlignRows(targetCorpus, new());

## Approach 1: save model then load it to get alignments

In [3]:
using System.IO;

Directory.CreateDirectory("out/VBL-WEB-FASTALIGN");
{
    using var srcTrgTrainer = new ThotWordAlignmentModelTrainer(ThotWordAlignmentModelType.FastAlign, parallelTextCorpus.Lowercase(),
        "out/VBL-WEB-FASTALIGN/src_trg");
    using var trgSrcTrainer = new ThotWordAlignmentModelTrainer(ThotWordAlignmentModelType.FastAlign,
        parallelTextCorpus.Invert().Lowercase(), "out/VBL-WEB-FASTALIGN/trg_src");
    using var symmetrizedTrainer = new SymmetrizedWordAlignmentModelTrainer(srcTrgTrainer, trgSrcTrainer);
    symmetrizedTrainer.Train(new DelegateProgress(status =>
        Console.WriteLine($"{status.Message}: {status.PercentCompleted:P}")));
    symmetrizedTrainer.Save();
    Console.WriteLine("Symmetrized FastAlign model saved");
}

Training direct alignment model: 0.00%
Training direct alignment model: 0.00%
Training direct alignment model: 8.33%
Training direct alignment model: 16.67%
Training direct alignment model: 25.00%
Training direct alignment model: 33.33%
Training direct alignment model: 41.67%
Training direct alignment model: 50.00%
Training inverse alignment model: 50.00%
Training inverse alignment model: 50.00%
Training inverse alignment model: 58.33%
Training inverse alignment model: 66.67%
Training inverse alignment model: 75.00%
Training inverse alignment model: 83.33%
Training inverse alignment model: 91.67%
Training inverse alignment model: 100.00%
Symmetrized FastAlign model saved


In [4]:
{
    using var srcTrgModel = new ThotFastAlignWordAlignmentModel("out/VBL-WEB-FASTALIGN/src_trg");
    using var trgSrcModel = new ThotFastAlignWordAlignmentModel("out/VBL-WEB-FASTALIGN/trg_src");
    using var symmetrizedModel = new SymmetrizedWordAlignmentModel(srcTrgModel, trgSrcModel)
    {
        Heuristic = SymmetrizationHeuristic.GrowDiagFinalAnd
    };
    foreach (ParallelTextRow textRow in parallelTextCorpus.Take(5))
    {
        var alignment = symmetrizedModel.GetBestAlignment(textRow.SourceSegment,
            textRow.TargetSegment);

        var verseRefStr = textRow.Ref.ToString();
        var sourceVerseText = string.Join(" ", textRow.SourceSegment);
        var targetVerseText = string.Join(" ", textRow.TargetSegment);
        Console.WriteLine(verseRefStr);
        Console.WriteLine($"Source: {sourceVerseText}");
        Console.WriteLine($"Target: {targetVerseText}");
        Console.WriteLine($"Alignment    : {alignment}");
    }
}


(10,25): error CS0103: The name 'symmetrizedModel' does not exist in the current context



Cell not executed: compilation error

## Approach 2: don't save model and get alignments

In [5]:
{
    using var srcTrgModel = new ThotFastAlignWordAlignmentModel();
    using var trainerSrcTrg = srcTrgModel.CreateTrainer(parallelTextCorpus.Lowercase());
    trainerSrcTrg.Train(new DelegateProgress(status =>
        Console.WriteLine($"Training FastAlign model: {status.PercentCompleted:P}")));
    trainerSrcTrg.Save();
    

    using var trgSrcModel = new ThotFastAlignWordAlignmentModel();
    using var trainerTrgSrc = trgSrcModel.CreateTrainer(parallelCorpus.Invert().Lowercase());
    trainerTrgSrc.Train(new DelegateProgress(status =>
        Console.WriteLine($"Training FastAlign model: {status.PercentCompleted:P}")));
    trainerTrgSrc.Save();

    
    using var symmetrizedModel = new SymmetrizedWordAlignmentModel(srcTrgModel, trgSrcModel)
    {
        Heuristic = SymmetrizationHeuristic.GrowDiagFinalAnd
    };
    foreach (ParallelTextRow textRow in parallelTextCorpus.Take(5))
    {
        var alignment = symmetrizedModel.GetBestAlignment(textRow.SourceSegment,
            textRow.TargetSegment);

        var verseRefStr = textRow.Ref.ToString();
        var sourceVerseText = string.Join(" ", textRow.SourceSegment);
        var targetVerseText = string.Join(" ", textRow.TargetSegment);
        Console.WriteLine(verseRefStr);
        Console.WriteLine($"Source: {sourceVerseText}");
        Console.WriteLine($"Target: {targetVerseText}");
        Console.WriteLine($"Alignment    : {alignment}");
    }
}



(3,57): error CS0103: The name 'parallelCorpus' does not exist in the current context

(10,57): error CS0103: The name 'parallelCorpus' does not exist in the current context

(22,25): error CS0103: The name 'symmetrizedModel' does not exist in the current context



Cell not executed: compilation error